# Classifying breast cancer molecular subtype using gene expression data


**In this exercise, we will using gene expression of 173 mutation driver genes associated with breast cancer to predict it's molecular subtype.**

[more description of the exercises]

In this exercise we'll [learn how to / use]:

- lasso regression
- random forest
- xgboost

[see examples in 'exercise' folder as template]

## Part 1: Set up and exploratory data analysis

In [ ]:
#Load the required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

#Load the data and perform exploratory data analysis
gene_exp = pd.read_csv("MTBC_Breast_Cancer.csv",sep='\t')

gene_exp.head(10)

In [ ]:
pd.pivot_table(gene_exp, values="Sample Identifier", index=["ER Status", "HER2 Status", "PR Status"], 
               columns=["Pam50 + Claudin-low subtype"], aggfunc='count', fill_value=0)

In [ ]:
#Check for any missing values 
gene_exp.isna().any().any()

In [ ]:
#Plots
#gene_data = gene_exp.drop(columns = ["ER Status", "HER2 Status", "PR Status","Pam50 + Claudin-low subtype"])

#gene_data = gene_data.T


## Part 2: Lasso regression

**Step 1. Prepare the data into x (predictors) and y (outcome) and training/test using 70/30 split**

In [ ]:
#Split the data into y and x variables (outcome and predictors)

#Rename the y variable column
gene_exp.rename(columns = {'Pam50 + Claudin-low subtype':'Cancer_Subtype'}, inplace=True)

#Converting the cancer subtype variable to categorical
gene_exp['Cancer_Subtype'] = gene_exp['Cancer_Subtype'].astype('category')

#Replace the claudin-low subtype to compatible formatting
gene_exp['Cancer_Subtype'] = gene_exp['Cancer_Subtype'].cat.rename_categories({'claudin-low':'claudinlow'})

gene_exp['Cancer_Subtype'].unique()

In [ ]:
#Create a variable containing outcome only
y=gene_exp['Cancer_Subtype']

#Create dummy variables for y as it contains multiple categories (this is easier for the algorithm to handle)
y = pd.get_dummies(y)
y.head()

In [ ]:
#Create dummy variables for categorical predictors
X_dummies = pd.get_dummies(gene_exp[['ER Status','HER2 Status','PR Status']])
X_dummies.head()

In [ ]:
#Drop the outcome, categorical predictors and sample identifier from the data
X_num = gene_exp.drop(['Cancer_Subtype','ER Status','HER2 Status','PR Status','Sample Identifier'], 
                      axis=1).astype('float64')

In [ ]:
#Create a list of numerical features
list_num = X_num.columns
list_num   #We can see there are 173 genes in the predictors list

In [ ]:
#Bring all the predictors together, note only one dummy variable column for each predictor is included
X = pd.concat([X_num, X_dummies[['ER Status_Positive','HER2 Status_Positive','PR Status_Positive']]], axis=1)
X.info()

In [ ]:
#Split data into train and test using a 70:30 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

X_train.head() #The gene expression data has already been stardadised so this step will not be repeated here

In [ ]:
y_train.head()

<br />
  
**Step 2. Using cross validation to find the best alpha (the penalty term in Lasso regression)**

<br />

In [ ]:
#Building the lasso model with cross validation to find optimal alpha
from sklearn.linear_model import MultiTaskLassoCV 

#Fitting Lasso with 10 fold cross validation using the best alpha
cv_lasso = MultiTaskLassoCV(cv=10, random_state=10, max_iter=10000).fit(X_train, y_train)

cv_lasso.alpha_  #This is the best alpha to use 

In [ ]:
#Creating the final lasso model incorporating best alpha from cv
from sklearn.linear_model import MultiTaskLasso

lasso_a = MultiTaskLasso(alpha = cv_lasso.alpha_)
lasso_a.fit(X_train, y_train)

In [ ]:
#Identify the most important features used in the lasso model
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(lasso_a, prefit=True)
X_new = model.transform(X_train)

In [ ]:
#Outputting the most important features with nonzero coefficients
feature_idx = model.get_support()
selected_features = X_train.columns[feature_idx]
print(selected_features)

#145 variables were used in the final model

<br />

**Step 3. Evaluating model performance**

<br />

In [ ]:
#Model evaluation 
from sklearn.metrics import r2_score

r2_train = r2_score(y_train, lasso_a.predict(X_train))
print("R squared training data:", round(r2_train*100, 2))

r2_test = r2_score(y_test, lasso_a.predict(X_test))
print("R squared test data:", round(r2_test*100, 2))

In [ ]:
#Mean squared error
from sklearn.metrics import mean_squared_error

mse_train = mean_squared_error(y_train, lasso_a.predict(X_train))
mse_test = mean_squared_error(y_test, lasso_a.predict(X_test))

print('Mean Squared Error of the training data:', round(mse_train, 2))
print('Mean Squared Error of the test data:', round(mse_test, 2))

<br />

**Visualising cross validation of alpha**

<br />

In [ ]:
#Plotting the results of cross-validation with mean squared error

plt.semilogx(cv_lasso.alphas_, cv_lasso.mse_path_, ":")
plt.plot(
    cv_lasso.alphas_ ,
    cv_lasso.mse_path_.mean(axis=-1),
    "k",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(
    cv_lasso.alpha_, linestyle="--", color="k", label="alpha: CV estimate"
)

plt.legend()
plt.xlabel("alphas")
plt.ylabel("Mean square error")
plt.title("Mean square error on each fold")
plt.axis("tight")

ymin, ymax = 0.04, 0.12
plt.ylim(ymin, ymax);

## Part 3:

## Part 4:

## Part x:

## Next steps

[provide suggested next steps for people who've completed the exercise]

Fill out the form below and we'll provide feedback on your code.

**Any feedback on the exercise? Any questions? Want feedback on your code? Please fill out the form [here](https://docs.google.com/forms/d/e/1FAIpQLSdoOjVom8YKf11LxJ_bWN40afFMsWcoJ-xOrKhMbfBzgxTS9A/viewform).**